### RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

/home/pal194/miniconda3/envs/py_rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 1 PDF files to process

Processing: Deep learning.pdf
  ✓ Loaded 1 pages

Total documents loaded: 1


In [3]:
all_pdf_documents

[Document(metadata={'producer': 'LibreOffice 7.3', 'creator': 'Writer', 'creationdate': '2025-12-01T15:43:44+10:00', 'author': 'User pc', 'source': '../data/pdf/Deep learning.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Deep learning.pdf', 'file_type': 'pdf'}, page_content='Deep  learning is a branch  of machine learning  that uses  multi-layered  artificial  neural  networks  to\nautomatically learn patterns and representations from data. Instead of relying on hand-crafted features, deep\nmodels  discover  useful  structures  directly  from  raw  inputs  like  images,  text,  or  audio  by  stacking  many\ntransformation layers. Early layers often learn simple patterns (edges, shapes, or word co-occurrences), while\ndeeper layers capture more abstract concepts (objects, semantics, or intent). This hierarchical representation\nlearning has enabled breakthroughs in computer vision, natural language processing, speech recognition, and\nmany other fields. However,

In [4]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [5]:
chunks=split_documents(all_pdf_documents)
chunks

Split 1 documents into 2 chunks

Example chunk:
Content: Deep  learning is a branch  of machine learning  that uses  multi-layered  artificial  neural  networks  to
automatically learn patterns and representations from data. Instead of relying on hand-craft...
Metadata: {'producer': 'LibreOffice 7.3', 'creator': 'Writer', 'creationdate': '2025-12-01T15:43:44+10:00', 'author': 'User pc', 'source': '../data/pdf/Deep learning.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Deep learning.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'LibreOffice 7.3', 'creator': 'Writer', 'creationdate': '2025-12-01T15:43:44+10:00', 'author': 'User pc', 'source': '../data/pdf/Deep learning.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Deep learning.pdf', 'file_type': 'pdf'}, page_content='Deep  learning is a branch  of machine learning  that uses  multi-layered  artificial  neural  networks  to\nautomatically learn patterns and representations from data. Instead of relying on hand-crafted features, deep\nmodels  discover  useful  structures  directly  from  raw  inputs  like  images,  text,  or  audio  by  stacking  many\ntransformation layers. Early layers often learn simple patterns (edges, shapes, or word co-occurrences), while\ndeeper layers capture more abstract concepts (objects, semantics, or intent). This hierarchical representation\nlearning has enabled breakthroughs in computer vision, natural language processing, speech recognition, and\nmany other fields. However,

### embedding And vectorStoreDB

In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
import torch
import numpy as np
from typing import List
from sentence_transformers import SentenceTransformer

class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2", device: str | None = None):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
            device: "cpu" or "cuda"; if None, auto-detect
        """
        self.model_name = model_name
        # auto-detect, but stay on CPU if CUDA isn't cleanly available
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name} on {self.device}")
            self.model = SentenceTransformer(self.model_name, device=self.device)
            print(
                f"Model loaded successfully. "
                f"Embedding dimension: {self.model.get_sentence_embedding_dimension()}"
            )
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        """
        if self.model is None:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts on {self.device}...")
        embeddings = self.model.encode(
            texts,
            show_progress_bar=True,
            device=self.device,  # be explicit here too
        )
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings

embedding_manager=EmbeddingManager()
embedding_manager

/home/pal194/miniconda3/envs/py_rag/lib/python3.10/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  return torch._C._cuda_getDeviceCount() > 0


Loading embedding model: all-MiniLM-L6-v2 on cpu
Model loaded successfully. Embedding dimension: 384


### VectorStore

In [9]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 4


In [10]:
chunks

[Document(metadata={'producer': 'LibreOffice 7.3', 'creator': 'Writer', 'creationdate': '2025-12-01T15:43:44+10:00', 'author': 'User pc', 'source': '../data/pdf/Deep learning.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Deep learning.pdf', 'file_type': 'pdf'}, page_content='Deep  learning is a branch  of machine learning  that uses  multi-layered  artificial  neural  networks  to\nautomatically learn patterns and representations from data. Instead of relying on hand-crafted features, deep\nmodels  discover  useful  structures  directly  from  raw  inputs  like  images,  text,  or  audio  by  stacking  many\ntransformation layers. Early layers often learn simple patterns (edges, shapes, or word co-occurrences), while\ndeeper layers capture more abstract concepts (objects, semantics, or intent). This hierarchical representation\nlearning has enabled breakthroughs in computer vision, natural language processing, speech recognition, and\nmany other fields. However,

In [11]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store in the vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 2 texts on cpu...


Batches: 100%|██████████| 1/1 [00:00<00:00, 20.48it/s]

Generated embeddings with shape: (2, 384)
Adding 2 documents to vector store...
Successfully added 2 documents to vector store
Total documents in collection: 6


### Retriever Pipeline From VectorStore

In [12]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)



In [13]:
rag_retriever

In [14]:
rag_retriever.retrieve("What is deep learning?")

Retrieving documents for query: 'What is deep learning?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts on cpu...


Batches: 100%|██████████| 1/1 [00:00<00:00, 22.06it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_d13a4fcb_1',
  'content': 'requirements, sensitivity to data quality and bias, and difficulties in interpretability and reliability—motivating\nongoing research into more efficient, robust, and trustworthy models.\nDeep learning is a powerful technique where computers learn to make decisions by passing data through\nmany layers of artificial neurons. Each layer transforms the input a little bit, gradually turning raw data—such\nas pixels, sound waves, or sensor readings—into meaningful features like shapes, words, or actions. By training\non large datasets, deep networks can recognize objects in images, translate between languages, drive cars, or\neven generate realistic text and images. Unlike traditional methods that require manual feature engineering,\ndeep  learning  automates  this  process,  often  achieving  superior  performance.  At  the  same  time,  it  raises\nimportant questions about transparency, fairness, and robustness, which are now central topics in mode

### RAG Pipeline- VectorDB To LLM Output Generation

In [15]:
import os
from dotenv import load_dotenv
load_dotenv()

print(os.getenv("GROQ_API_KEY"))

None


In [16]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

In [17]:
class GroqLLM:
    def __init__(self, model_name: str = "llama-3.3-70b-versatile", api_key: str =None):
        """
        Initialize Groq LLM
        
        Args:
            model_name: Groq model name (qwen2-72b-instruct, llama3-70b-8192, etc.)
            api_key: Groq API key (or set GROQ_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        print(self.api_key)
        
        if not self.api_key:
            raise ValueError("Groq API key is required. Set GROQ_API_KEY environment variable or pass api_key parameter.")
        
        self.llm = ChatGroq(
            groq_api_key=self.api_key,
            model_name=self.model_name,
            temperature=0.1,
            max_tokens=1024
        )
        
        print(f"Initialized Groq LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context
        
        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length
            
        Returns:
            Generated response string
        """
        
        # Create prompt template
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
        )
        
        # Format the prompt
        formatted_prompt = prompt_template.format(context=context, question=query)
        
        try:
            # Generate response
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content
            
        except Exception as e:
            return f"Error generating response: {str(e)}"
        
    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting
        
        Args:
            query: User question
            context: Retrieved context
            
        Returns:
            Generated response
        """
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""
        
        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"
    


In [1]:
# Initialize Groq LLM (you'll need to set GROQ_API_KEY environment variable)
try:
    # groq_llm = GroqLLM(api_key=os.getenv("GROQ_API_KEY"))
    groq_llm = GroqLLM(api_key="provie_key")
    print("Groq LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your GROQ_API_KEY environment variable to use the LLM.")
    groq_llm = None

NameError: name 'GroqLLM' is not defined

In [19]:
### get the context from the retriever and pass it to the LLM

rag_retriever.retrieve("Unified Multi-task Learning Framework")

Retrieving documents for query: 'Unified Multi-task Learning Framework'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts on cpu...


Batches: 100%|██████████| 1/1 [00:00<00:00, 142.80it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)


[]

### Integration Vectordb Context pipeline With LLM output

In [20]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.3-70b-versatile",temperature=0.1,max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [21]:
answer=rag_simple("What is Deep learning?",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is Deep learning?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts on cpu...


Batches: 100%|██████████| 1/1 [00:00<00:00, 144.48it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


Deep learning is a powerful technique where computers learn to make decisions by passing data through many layers of artificial neurons, transforming raw data into meaningful features.


### Enhanced RAG Pipeline Features

In [22]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("Hard Negative Mining Technqiues", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'Hard Negative Mining Technqiues'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts on cpu...


Batches: 100%|██████████| 1/1 [00:00<00:00, 141.12it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)
Answer: No relevant context found.
Sources: []
Confidence: 0.0
Context Preview: 


In [24]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("what is deep learning", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'what is deep learning'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts on cpu...


Batches: 100%|██████████| 1/1 [00:00<00:00, 102.10it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
requirements, sensitivity to data quality and bias, and difficulties in interpretability and reliability—motivating
ongoing research into more efficient, robust, and trustworthy models.
Deep learning is a powerful technique where computers learn to ma

ke decisions by passing data through
many layers of artificial neurons. Each layer transforms the input a little bit, gradually turning raw data—such
as pixels, sound waves, or sensor readings—into meaningful features like shapes, words, or actions. By training
on large datasets, deep networks can recognize objects in images, translate between languages, drive cars, or
even generate realistic text and images. Unlike traditional methods that require manual feature engineering,
deep  learning  automates  this  process,  often  achieving  superior  performance.  At  the  same  time,  it  raises
important questions about transparency, fairness, and robustness, which are now central topics in modern AI
research.
Page 1 of 1

requirements, sensitivity to data quality and bias, and difficulties in interpretability and reliability—motivating
ongoing research into more efficient, robust, and trustworthy models.
Deep learning is a powerful technique where computers learn to make decisions by pas